In [262]:
import os
import sys
import re
import numpy as np 
import pandas as pd
from pathlib import Path
import matplotlib as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')

dir_meta = Path('meta')
dir_meta.mkdir(exist_ok=True, parents=True)

print(pd.__version__)
print(np.__version__)

0.25.3
1.19.5



#### 讀取資料

#### 資料格式對齊:
- 資料過濾
    - 刪除沒有資料的欄位 
        - df.dropna(axis='columns',how='all', inplace=True)
    - 刪除沒有名稱的欄位 
        - df.loc[:, ~df_CA.columns.str.contains('^Unnamed')]
    - 刪除沒有檔名的資料 
        - df.dropna(subset=['No.'], how='any')
    - 刪除欄位名稱是超連結的欄位 
        - df.loc[:, ~df_CA.columns.str.contains('https:')]
- 資料轉換
    - 欄位名稱統一
        - df.rename(columns={'ID_20210210':'Species', 'N_W_length':'W_length',	'N_B_length':'B_length'})
    - 增加Event欄位
        - df['No.'].apply(lambda str_: str_[:4])
    - 改變資料型態
        - 物件 > 浮點數
            -  df[['Dw(g)','W_length','B_length']].apply(pd.to_numeric, errors='coerce')

In [184]:
def excel_file_preprocess(df:pd.DataFrame)->pd.DataFrame:
    df.dropna(axis='columns',how='all', inplace=True)
    df.dropna(subset=['No.'], how='any', inplace=True)
    df['Event'] = df['No.'].apply(lambda str_: str_[:4])
    print(f'Datt size : {len(df_CA)}. | # of Cols : {len(df_CA.columns)}')

    df.rename(columns={'ID_20210210':'Species', 'N_W_length':'W_length', 'N_B_length':'B_length'}, inplace=True)

    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.loc[:, ~df.columns.str.contains('https:')]
    df[['Dw(g)','W_length','B_length']] = df[['Dw(g)','W_length','B_length']].apply(pd.to_numeric, errors='coerce')
    return df

In [ ]:
# df_CA = pd.read_excel('MC-SJ-CA_ThermalTolerance-Range Size.xlsx',
#                       header=0,
#                       skiprows=list(range(1,16)),
#               sheet_name='CATT&RS')

# df_CA = excel_file_preprocess(df_CA)
# df_CA


In [189]:
# df_CA = pd.read_csv('CA.csv', index_col=0)
# col_sel = ['Event', 'No.', 'Family', 'Species', 'Sex', 'Altitude', 'Hot', 'Cold', 'Temp_avg', 'Dw(g)', 'W_length', 'B_length']
# df_CA = df_CA[col_sel]
# df_CA.to_csv('CA.csv')
df_CA = pd.read_csv('CA.csv', index_col=0)
print(f'Datt size : {len(df_CA)}. | # of Cols : {len(df_CA.columns)}')

col_CA = df_CA.columns
print(col_CA)
# print(df_CA.info())
print(df_CA.describe())
df_CA

Datt size : 4356. | # of Cols : 12
Index(['Event', 'No.', 'Family', 'Species', 'Sex', 'Altitude', 'Hot', 'Cold',
       'Temp_avg', 'Dw(g)', 'W_length', 'B_length'],
      dtype='object')
          Altitude          Hot         Cold     Temp_avg        Dw(g)  \
count  4316.000000  2072.000000  2071.000000  2075.000000  4306.000000   
mean   1132.088508     0.523649     0.476581    22.874072     0.020842   
std     455.417129     0.499561     0.499572    19.638023     0.041907   
min     140.000000     0.000000     0.000000    -5.600000    -0.332300   
25%     796.000000     0.000000     0.000000     2.400000     0.004200   
50%    1066.000000     1.000000     0.000000    36.600000     0.009500   
75%    1374.000000     1.000000     1.000000    41.900000     0.021800   
max    1959.000000     1.000000     1.000000    47.500000     0.710600   

          W_length     B_length  
count  4232.000000  4303.000000  
mean     13.951276    11.796235  
std       5.914649     4.646003  
min      

,Event,No.,Family,Species,Sex,Altitude,Hot,Cold,Temp_avg,Dw(g),W_length,B_length
0,CATT,CATT2131,Hypeninae,CA_NAA01,f,509.0,1.0,0.0,44.7,0.00150,8.19,6.89
1,CATT,CATT2130,Hypeninae,CA_NAA02,m,509.0,0.0,1.0,-1.9,0.00050,6.16,5.46
2,CATT,CATT2129,Lasiocampidae,CA_LAS04,m,509.0,1.0,0.0,43.9,0.07010,19.14,19.30
3,CATT,CATT2128,Lasiocampidae,CA_LAS04,m,509.0,0.0,1.0,8.4,0.08970,19.27,23.56
4,CATT,CATT2127,Erebidae,CA_HER07,m,509.0,1.0,0.0,39.5,0.00290,9.13,7.44
...,...,...,...,...,...,...,...,...,...,...,...,...
4351,CARS,CARS0005,Geometridae,CA_GEO47,NaN,1066.0,NaN,NaN,NaN,0.00523,12.30,9.24
4352,CARS,CARS0004,Geometridae,CA_GEO106,NaN,1066.0,NaN,NaN,NaN,0.01378,12.34,11.42
4353,CARS,CARS0003,Geometridae,CA_RSGEO152,NaN,1066.0,NaN,NaN,NaN,0.00528,13.35,10.83
4354,CARS,CARS0002,Crambidae,CA_CRA02,NaN,1066.0,NaN,NaN,NaN,0.01720,13.00,12.82


In [ ]:
# df_SJ = pd.read_excel('MC-SJ-CA_ThermalTolerance-Range Size.xlsx',
#                       index_col=0,
#                       header=0,
#                     #   skiprows=list(range(1,16)),
#               sheet_name='SJTT&RS')
# df_SJ = excel_file_preprocess(df_SJ)
# df_SJ

In [191]:
# col_sel = ['Event', 'No.', 'Family', 'Species', 'Sex', 'Altitude', 'Hot', 'Cold', 'Temp_avg', 'Dw(g)', 'W_length', 'B_length']
# df_SJ = df_SJ[col_sel]
# df_SJ.to_csv('SJ.csv')
df_SJ = pd.read_csv('SJ.csv', index_col=0)
print(f'Datt size : {len(df_CA)}. | # of Cols : {len(df_CA.columns)}')

col_SJ = df_SJ.columns
print(col_SJ)
# print(df_SJ.info())
print(df_SJ.describe())
df_SJ

Datt size : 4356. | # of Cols : 12
Index(['Event', 'No.', 'Family', 'Species', 'Sex', 'Altitude', 'Hot', 'Cold',
       'Temp_avg', 'Dw(g)', 'W_length', 'B_length'],
      dtype='object')
          Altitude          Hot         Cold     Temp_avg        Dw(g)  \
count  4274.000000  2247.000000  2247.000000  2261.000000  4207.000000   
mean   2341.643659     0.526480     0.473520    22.446351     0.043066   
std     941.083010     0.499409     0.499409    20.723560     0.061030   
min     860.000000     0.000000     0.000000    -7.000000    -0.406900   
25%    1864.000000     0.000000     0.000000     0.700000     0.011870   
50%    2000.000000     1.000000     0.000000    36.900000     0.026800   
75%    3147.000000     1.000000     1.000000    42.300000     0.051500   
max    4152.000000     1.000000     1.000000    48.900000     0.866580   

          W_length     B_length  
count  4083.000000  4015.000000  
mean     18.326809    14.660134  
std       4.924894     4.797497  
min      

,Event,No.,Family,Species,Sex,Altitude,Hot,Cold,Temp_avg,Dw(g),W_length,B_length
1,SJTT,SJTT0001,Erebidae,SJ_LIT02,female,2000.0,1.0,0.0,37.7,0.01180,18.98,11.53
2,SJTT,SJTT0002,Erebidae,SJ_LIT09,female,2000.0,1.0,0.0,38.8,0.04600,23.65,14.74
3,SJTT,SJTT0003,Notodontidae,SJ_NOT02,female,2000.0,1.0,0.0,42.1,0.11740,32.80,24.75
4,SJTT,SJTT0004,Erebidae,SJ_LIT21,male,2000.0,0.0,1.0,4.0,0.00560,13.69,10.24
5,SJTT,SJTT0005,Erebidae,SJ_HYP06,male,2000.0,1.0,0.0,39.7,0.01790,18.82,15.38
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,SJRS,SJRS00030,Noctuidae,SJ_NOC088,female,4152.0,NaN,NaN,NaN,0.04022,14.44,12.82
4273,SJRS,SJRS00025,Noctuidae,SJ_NOC109,male,4152.0,NaN,NaN,NaN,0.03843,15.29,14.72
4274,SJRS,SJRS00003,Noctuidae,SJ_NOC126,male,4152.0,NaN,NaN,NaN,0.02719,12.79,11.72
4245,SJRS,SJRS00018,Geometridae,NaN,female,4152.0,NaN,NaN,NaN,0.00620,14.10,10.36


- 檔案命名混亂
    - 以正規表達式抓取檔名來跟METADATA作關聯

#### 資量匯整合併

In [193]:
df = pd.concat([df_SJ, df_SJ])
df.describe()

,Altitude,Hot,Cold,Temp_avg,Dw(g),W_length,B_length
count,8548.000000,4494.000000,4494.000000,4522.000000,8414.000000,8166.000000,8030.000000
mean,2341.643659,0.526480,0.473520,22.446351,0.043066,18.326809,14.660134
std,941.027955,0.499354,0.499354,20.721268,0.061026,4.924592,4.797198
min,860.000000,0.000000,0.000000,-7.000000,-0.406900,8.710000,6.520000
25%,1864.000000,0.000000,0.000000,0.700000,0.011860,15.190000,11.530000
50%,2000.000000,1.000000,0.000000,36.900000,0.026800,17.620000,13.890000
75%,3147.000000,1.000000,1.000000,42.300000,0.051500,20.650000,16.777500
max,4152.000000,1.000000,1.000000,48.900000,0.866580,54.600000,52.620000


In [196]:
df

,Event,No.,Family,Species,Sex,Altitude,Hot,Cold,Temp_avg,Dw(g),W_length,B_length
1,SJTT,SJTT0001,Erebidae,SJ_LIT02,female,2000.0,1.0,0.0,37.7,0.01180,18.98,11.53
2,SJTT,SJTT0002,Erebidae,SJ_LIT09,female,2000.0,1.0,0.0,38.8,0.04600,23.65,14.74
3,SJTT,SJTT0003,Notodontidae,SJ_NOT02,female,2000.0,1.0,0.0,42.1,0.11740,32.80,24.75
4,SJTT,SJTT0004,Erebidae,SJ_LIT21,male,2000.0,0.0,1.0,4.0,0.00560,13.69,10.24
5,SJTT,SJTT0005,Erebidae,SJ_HYP06,male,2000.0,1.0,0.0,39.7,0.01790,18.82,15.38
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,SJRS,SJRS00030,Noctuidae,SJ_NOC088,female,4152.0,NaN,NaN,NaN,0.04022,14.44,12.82
4273,SJRS,SJRS00025,Noctuidae,SJ_NOC109,male,4152.0,NaN,NaN,NaN,0.03843,15.29,14.72
4274,SJRS,SJRS00003,Noctuidae,SJ_NOC126,male,4152.0,NaN,NaN,NaN,0.02719,12.79,11.72
4245,SJRS,SJRS00018,Geometridae,NaN,female,4152.0,NaN,NaN,NaN,0.00620,14.10,10.36


In [200]:
df.query('B_length.isnull()')

,Event,No.,Family,Species,Sex,Altitude,Hot,Cold,Temp_avg,Dw(g),W_length,B_length
46,SJTT,SJTT0046,Geometridae,SJ_GEO063,NaN,2000.0,1.0,0.0,39.4,0.01220,20.51,NaN
75,SJTT,SJTT0075,Erebidae,SJ_LIT11,female,2000.0,1.0,0.0,41.4,NaN,13.73,NaN
104,SJTT,SJTT0104,Geometridae,SJ_GEO142,NaN,2000.0,1.0,0.0,42.2,NaN,17.98,NaN
135,SJTT,SJTT0135,Noctuidae,SJ_NOC063,NaN,2000.0,NaN,NaN,NaN,NaN,NaN,NaN
138,SJTT,SJTT0138,Geometridae,SJ_GEO136,NaN,2000.0,1.0,0.0,41.6,NaN,17.91,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4098,SJRS,SJRS01718,NaN,NaN,NaN,3579.0,NaN,NaN,NaN,0.06740,NaN,NaN
4100,SJRS,SJRS00437,Geometridae,NaN,male,3800.0,NaN,NaN,NaN,0.00587,NaN,NaN
4243,SJRS,SJRS01716,NaN,NaN,NaN,3800.0,NaN,NaN,NaN,0.04950,NaN,NaN
4244,SJRS,SJRS01717,NaN,NaN,NaN,3800.0,NaN,NaN,NaN,0.07490,NaN,NaN


- 檢視整理好的、species-based 資料

In [206]:
df_sp_based = pd.read_csv('210607_CSV_All_Species_based.csv')
df_sp_based

,Location,Family,Species,Sample Size,TTRS_Elev_up,TTRS_Elev_low,TTRS_Elev_mid,TTRS_RS,TTRS_W_length,TTRS_B_length,TTRS_Weight,TTRS_CTmax,TTRS_CTmin,TTRS_CTrange,TTRS_Tmax,TTRS_Tmin,TTRS_Tmean,TTRS_DTR
0,Malaysia,Geometridae,CA_Abaxas_sp2,2,1959,1959,1959.0,0,28.025000,17.925000,0.027650,35.100000,0.100000,35.000000,17.298785,13.297546,15.040596,4.001239
1,Malaysia,Erebidae,CA_AGA01,2,1372,509,940.5,863,31.835000,22.700000,0.139000,41.000000,10.900000,30.100000,23.906834,18.108418,20.623503,5.798416
2,Malaysia,Erebidae,CA_AGA02,2,1066,903,984.5,163,31.390000,22.430000,0.094690,41.700000,NaN,NaN,23.963156,19.042622,21.094452,4.920534
3,Malaysia,Erebidae,CA_AGA03,1,1800,1800,1800.0,0,32.450000,24.520000,0.145590,NaN,NaN,NaN,18.717044,14.311935,16.155921,4.405110
4,Malaysia,Erebidae,CA_AMATA01,17,1374,903,1138.5,471,14.821875,12.638235,0.011705,41.800000,2.600000,39.200000,23.575832,18.442713,20.619982,5.133119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,China,Drepanidae,SJ_THY15,32,3579,2000,2789.5,1579,17.041875,12.862187,0.026804,38.342857,-1.976923,40.319780,21.146612,12.433047,15.682796,8.713565
3174,China,Drepanidae,SJ_THY16,7,2856,1864,2360.0,992,19.010000,15.688571,0.054564,NaN,-1.450000,NaN,20.522450,10.974509,14.733828,9.547941
3175,China,Drepanidae,SJ_THY17,4,2856,2000,2428.0,856,19.702500,15.540000,0.061695,40.600000,4.100000,36.500000,20.047220,10.660688,14.423787,9.386532
3176,China,Uraniidae,SJ_URA01,13,3175,1864,2519.5,1311,13.691667,9.000833,0.008242,39.166667,-1.200000,40.366667,20.598435,10.588436,14.490011,10.009999


In [204]:
df_sp_based.groupby('Location').mean()

,Sample Size,TTRS_Elev_up,TTRS_Elev_low,TTRS_Elev_mid,TTRS_RS,TTRS_W_length,TTRS_B_length,TTRS_Weight,TTRS_CTmax,TTRS_CTmin,TTRS_CTrange,TTRS_Tmax,TTRS_Tmin,TTRS_Tmean,TTRS_DTR
Location,,,,,,,,,,,,,,,
China,6.095238,2264.080357,1791.279762,2027.68006,472.800595,17.891089,14.309312,0.040103,42.124513,1.043026,41.207814,24.052720,14.644782,18.080844,9.407938
Malaysia,2.732166,1162.319066,994.237354,1078.27821,168.081712,13.120170,11.364688,0.019359,41.461537,2.737338,38.445990,23.199600,17.882645,20.171556,5.316955
Taiwan,12.023859,1858.044606,1346.986515,1602.51556,511.058091,18.245272,14.951575,0.057042,41.332484,1.809045,39.407495,23.964022,16.330847,19.326446,7.633176


In [210]:
df_taiwan = df_sp_based.query('Location == "Taiwan"')
print(df_taiwan.Species.nunique())
df_taiwan.Species.unique()

964


array(['"Adrapsa" sp.', '"Collix" sp.', '"Eilema" sp. 1',
       '"Eilema" sp. 2', '"Eilema" sp. 3', '"Koyaga" sp._blue',
       '"Lophophleps" sp.', '"Plecoptera" sp.', '"Rivula" sp.',
       '"Syncosmia" sp.', 'Abaciscus costimacula',
       'Abaciscus tristis tristis', 'Aberrasine lichenshihi',
       'Abraxas adilluminata', 'Abraxas consputa', 'Abraxas illuminata',
       'Abraxas persimplex', 'Abraxas placata', 'Abraxas submartiaria',
       'Abraxas suspecta', 'Abraxas tenellula', 'Abraxas tenuisuffusa',
       'Abraxas wilemani', 'Abrostola suisharyonis suisharyonis',
       'Acasis viretata himalayica', 'Achrosis rufescens',
       'Acolutha pictaria imbecilla', 'Acolutha pulchella semifulva',
       'Acosmetia chinensis', 'Acronicta gigasa', 'Acronicta hercules',
       'Acronicta intermedia', 'Acronicta pruinosa', 'Acronicta sp.',
       'Actias ningpoana ningtaiwana', 'Adrapsa incertalis',
       'Adrapsoides reticulatis', 'Aedia acronyctoides', 'Aelidae sp.',
       'Aeolan

#### 根據影像檔名抓出對應的物種代號
- CA
- SJ
- MC

In [261]:
dir_imgs = Path('../origin')
paths_imgs = list(dir_imgs.glob('*.png'))
print(len(paths_imgs))
fnames_imgs = [path.stem for path in paths_imgs]
df_imgs = pd.DataFrame(fnames_imgs, columns=['fname'])
df_imgs

8921


,fname
0,Aberrasine lichenshihi_male_FBG_10-IV-2015_SWu...
1,"Abraxas adilluminata Inoue, 1984_male_01_cropped"
2,Abraxas adilluminata_female_Biluxi_1-VII-2015_...
3,Abraxas adilluminata_female_Hewang_28-X-2015_d...
4,Abraxas adilluminata_female_MTTS1558_1_cropped
...,...
8916,Xestia supravidua_male_MCE2750_24-25-VII-2015_...
8917,Xestia supravidua_PT_female_Taiwan_800px_cropped
8918,Xestia tamsi_NSMT2015_180_1_800_cropped
8919,Yashmakia suffusa_male_Wulai_15-III-2014_SWu&W...


In [231]:
df_origin = pd.read_csv('origin.csv',sep='\t', header=None,  names=['fname'])
df_origin['fname'] = df_origin.fname.str.replace('.png', '')
df_origin


,fname
0,Aberrasine lichenshihi_male_FBG_10-IV-2015_SWu...
1,Abraxas adilluminata_female_Biluxi_1-VII-2015_...
2,Abraxas adilluminata_female_Hewang_28-X-2015_d...
3,Abraxas adilluminata_female_MTTS1558_1_cropped
4,Abraxas adilluminata_female_MTTS302_1_cropped
...,...
8916,Xestia supravidua_male_MCE2750_24-25-VII-2015_...
8917,Xestia supravidua_PT_female_Taiwan_800px_cropped
8918,Xestia tamsi_NSMT2015_180_1_800_cropped
8919,Yashmakia suffusa_male_Wulai_15-III-2014_SWu&W...


- CA&SJ

In [281]:
idx = df_imgs.query('fname.str.contains("CA") or fname.str.contains("SJ")').index.values

In [296]:
df_imgs_ = df_imgs.iloc[idx].copy()

In [286]:
pattern = "SJ|CA{1}TT|RS{1}[\d]*"
regex = re.compile()
df_imgs.iloc[idx].fname.apply(lambda x : regex.search(x))

85      <_sre.SRE_Match object; span=(6, 8), match='SJ'>
86      <_sre.SRE_Match object; span=(6, 8), match='SJ'>
87      <_sre.SRE_Match object; span=(6, 8), match='SJ'>
88      <_sre.SRE_Match object; span=(6, 8), match='SJ'>
89      <_sre.SRE_Match object; span=(6, 8), match='SJ'>
                              ...                       
8901    <_sre.SRE_Match object; span=(6, 8), match='SJ'>
8902    <_sre.SRE_Match object; span=(6, 8), match='SJ'>
8903    <_sre.SRE_Match object; span=(6, 8), match='SJ'>
8904    <_sre.SRE_Match object; span=(6, 8), match='SJ'>
8905    <_sre.SRE_Match object; span=(6, 8), match='SJ'>
Name: fname, Length: 8346, dtype: object

In [298]:
pattern = "SJTT[\d]*|CATTT[\d]*|CARS[\d]*|SJRS[\d]*"
df_imgs_.fname.str.findall(pattern)

85       [SJTT0564]
86       [SJTT0586]
87       [SJTT0925]
88      [SJRS00701]
89      [SJRS00733]
           ...     
8901     [SJTT0713]
8902     [SJTT0747]
8903     [SJTT0747]
8904     [SJTT0812]
8905     [SJTT1242]
Name: fname, Length: 8346, dtype: object

- MC